In [ ]:
{-# LANGUAGE FunctionalDependencies #-}
{-# LANGUAGE MonomorphismRestriction #-}
{-# LANGUAGE FlexibleContexts #-}

import Control.Monad.State
import Numeric.LinearAlgebra
import Numeric.LinearAlgebra.Data
import Numeric
import Prelude hiding ((<>))

\begin{eqnarray}
Sigmoid \nonumber \\
  h(x) &=& \frac
  {1}
  {1 + \exp(-x)} \\
\nonumber \\
ReLU \nonumber \\
  h(x) &=& \begin{cases}
  x & (x > 0) \\
  0 & (x \le 0)
  \end{cases}
\end{eqnarray}

In [ ]:
sigmoid :: (Floating a) => a -> a
sigmoid a = fromIntegral 1 / (fromIntegral 1 + exp (-a))

sigmoidm :: (Floating a, Container Matrix a) => Matrix a -> Matrix a
sigmoidm = cmap sigmoid

sigmoidBackward :: (Floating a, Num (Vector a), Container Matrix a) => Matrix a -> Matrix a -> Matrix a
sigmoidBackward y d = d * (cmap (\a -> fromIntegral 1 - a) y) * y

relu :: (Ord a, Num a) => a -> a
relu = max $ fromIntegral 0

relum :: (Ord a, Num a, Container Matrix a) => Matrix a -> Matrix a
relum = cmap relu

relumBackward :: (Ord a, Num a, Num (Matrix a), Container Vector a, Container Matrix a) => Matrix a -> Matrix a -> Matrix a
relumBackward x d = d * mask
    where mask = cmap (\a -> fromIntegral $ if (0 < a) then 1 else 0) x

\begin{eqnarray}
Cross Entropy \nonumber \\
  L &=& -\displaystyle \sum_{i=1}^{n}t_i \log (y_i + d) \\
  t &:& 教師ラベル \nonumber \\
  d &:& 微小値 \nonumber \\
 \nonumber \\
Softmax \nonumber \\
  y_k &=& \frac
  {\exp(a_k - \hat{a})}
  {\displaystyle \sum_{i=1}^{n}\exp(a_i - \hat{a})} \\
  \hat{a} &=& \max \{ a_{1...n} \} \nonumber
\end{eqnarray}


In [ ]:
softmax :: (Floating a, Container Vector a) => Vector a -> Vector a
softmax v = cmap (/s) v'
    where
    m = maxElement v
    v' = cmap (\a -> exp (a - m)) v
    s = sumElements v'

softmaxm :: (Floating a, Container Vector a) => Matrix a -> Matrix a
softmaxm m = fromRows $ map softmax $ toRows m

crossEntropy :: (Floating a, Num (Vector a), Container Vector a) => Vector a -> Vector a -> a
crossEntropy t y = -(sumElements $ cmap log (y + d) * t)
    where d = 1e-10

crossEntropym :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
crossEntropym t m = sum vs / batchSize
    where 
    vs = uncurry crossEntropy `map` (toRows t `zip` toRows m)
    batchSize = fromIntegral $ rows t

softmaxWithCross :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> a
softmaxWithCross t = crossEntropym t . softmaxm

softmaxWithCrossBackward :: (Floating a, Num (Vector a), Container Vector a) => Matrix a -> Matrix a -> Matrix a
softmaxWithCrossBackward t y = (y - t) / batchSize
    where batchSize = fromIntegral $ rows t

In [ ]:
affinem :: (Floating a, Numeric a, Num (Vector a)) => Matrix a -> Vector a -> Matrix a -> Matrix a
affinem w b x = x <> w + b'
    where b' = fromColumns $ replicate (rows x) b

affinemBackward :: (Floating a, Numeric a) => Matrix a -> Matrix a -> Matrix a -> (Matrix a, Matrix a, Vector a)
affinemBackward w x d = (dx, dw, db)
    where
    dx = d <> tr w
    dw = tr x <> d
    db = fromList $ sumElements `map` toColumns d

In [ ]:
class ForwardLayer a where
    forward :: (Ord v, Floating v, Numeric v, Num (Vector v), BackwardLayer b) => a -> Matrix v -> (b, Matrix v)

class BackwardLayer a where
    backward :: (Ord v, Floating v, Numeric v, Num (Vector v), ForwardLayer b) => a -> Matrix v -> (b, Matrix v)

class OutputLayer a where
    output :: (Floating v, Numeric v, Num (Vector v), BackputLayer b) => a -> Matrix v -> Matrix v -> (b, v)

class BackputLayer a where
    backput :: (Floating v, Numeric v, Num (Vector v), OutputLayer b) => a -> (b, Matrix v)

In [ ]:
data AffineForward a = AffineForward (Matrix a) (Vector a)
data SigmoidForward
data ReLUForward
data SoftmaxWithCrossForward

data AffineBackward a = AffineBackward (Matrix a) (Matrix a)
data SigmoidBackward y
data ReLUBackward x
data SoftmaxWithCrossBackward t

In [ ]:
instance ForwardLayer (AffineForward a) where
    forward (AffineForward w b) x = (AffineBackward w x, r)
        where r = affinem w b x

In [ ]:
data LearningLayers a = OutputLayer a | ForwardingLayers a (LearningLayers a)

In [ ]:
xs = ((2><2)[1..]) :: Matrix R
ys = ((2><2)[1..]) :: Matrix R
:type xs
:type ys
print xs
print ys
print $ xs / 2.0